In [13]:
import nest_asyncio
nest_asyncio.apply()

import asyncio

from neo4j import GraphDatabase
from neo4j_graphrag.embeddings import OpenAIEmbeddings
from neo4j_graphrag.experimental.pipeline.kg_builder import SimpleKGPipeline
from neo4j_graphrag.llm import OpenAILLM


In [14]:
NEO4J_URI = "neo4j://localhost:7687"
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "cassandra"

In [45]:
from neo4j import GraphDatabase
from neo4j_graphrag.indexes import create_vector_index

NEO4J_URI = "neo4j://localhost:7687"
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "cassandra"
INDEX_NAME = "yomap-index"

# Connect to the Neo4j database
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

# Create the index
create_vector_index(
    driver,
    INDEX_NAME,
    label="Chunk",
    embedding_property="embedding",
    dimensions=3072,
    similarity_fn="euclidean",
)
driver.close()

In [41]:
from neo4j import GraphDatabase
from neo4j_graphrag.embeddings import OpenAIEmbeddings
from neo4j_graphrag.indexes import upsert_vectors
from neo4j_graphrag.types import EntityType

NEO4J_URI = "neo4j://localhost:7687/yomap"
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "cassandra"

# Connect to the Neo4j database
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

# Create an Embedder object
embedder = OpenAIEmbeddings(model="text-embedding-3-large")

# Generate an embedding for some text
text = (
    "The son of Duke Leto Atreides and the Lady Jessica, Paul is the heir of House "
    "Atreides, an aristocratic family that rules the planet Caladan."
)
vector = embedder.embed_query(text)

# Upsert the vector
upsert_vectors(
    driver,
    ids=["1234"],
    embedding_property="vectorProperty",
    embeddings=[vector],
    entity_type=EntityType.NODE,
)
driver.close()

In [42]:
# Connect to the Neo4j database
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

# List the entities and relations the LLM should look for in the text
entities = ["Person", "House", "Planet"]
relations = ["PARENT_OF", "HEIR_OF", "RULES"]
potential_schema = [
    ("Person", "PARENT_OF", "Person"),
    ("Person", "HEIR_OF", "House"),
    ("House", "RULES", "Planet"),
]

# Create an Embedder object
embedder = OpenAIEmbeddings(model="text-embedding-3-large")

# Instantiate the LLM
llm = OpenAILLM(
    model_name="gpt-4o",
    model_params={
        "max_tokens": 2000,
        "response_format": {"type": "json_object"},
        "temperature": 0,
    },
)

# Instantiate the SimpleKGPipeline
kg_builder = SimpleKGPipeline(
    llm=llm,
    driver=driver,
    embedder=embedder,
    entities=entities,
    relations=relations,
    on_error="IGNORE",
    from_pdf=False,
)

# Run the pipeline on a piece of text
text = (
    "The son of Duke Leto Atreides and the Lady Jessica, Paul is the heir of House "
    "Atreides, an aristocratic family that rules the planet Caladan."
)

In [43]:
asyncio.run(kg_builder.run_async(text=text))
driver.close()

# Get data from Elastic

In [11]:
from datetime import datetime
from elasticsearch import Elasticsearch

In [12]:
client = Elasticsearch(
    hosts=["https://yomap-elastic-dev.fly.dev"],
    basic_auth=("yomapsearch", "@6e9qxFPQD?@dYPk"),
)

In [13]:
resp = client.search(index="profiles", query={"match_all": {}})

In [41]:
paragraph = []

for profile in resp["hits"]["hits"]:
    print(profile["_source"])

    address = profile["_source"]["address"]
    about = profile["_source"]["about"]
    name = profile["_source"]["displayName"]
    age = profile["_source"]["age"] if profile["_source"]["age"] is not None else 0
    service = profile["_source"]["service"]
    tags = profile["_source"]["tags"]

    paragraph.append("name: " + name + " address: " + address + " about_me: " + about + " age: " + str(age) + " category: " + str(service) + " tags: " + str(tags))

{'address': 'Panamá, , Panamá, Panama', 'gender': '', 'displayName': 'Arizza', 'rating': 0, 'about': '', 'photo': '', 'blockedProfiles': [], 'userId': 'DGnrQapGUTPhGJ6Noq2GuuDCzM82', 'totalRatingVotes': 0, 'network': [], 'tags': [{'weight': 0, 'id': 'pf8IUMBw9rVBMGHoHdYw', 'text': 'user', 'slug': 'user', 'parentId': '', 'usedBy': 14, 'parentSlug': ''}], 'service': {'weight': 0, 'id': 'pf8IUMBw9rVBMGHoHdYw', 'text': 'user', 'slug': 'user', 'parentId': '', 'usedBy': 14, 'parentSlug': ''}, 'commentsCount': 0, 'socialNetworks': [], 'location': {'lat': 8.994300842285156, 'lon': -79.55968186016943}, 'age': None, 'realTimeLocEnabled': False, 'profileId': '32z0evf2wJBkKNMaZYql'}
{'address': 'PQ5V+3PR, PQ5V+3PR, Punto Fijo, Falcón, Venezuela', 'gender': 'female', 'realTimeLocEnabled': False, 'displayName': 'Oriana Vega24422', 'about': '', 'photo': 'Files/Users/scXuutTfVsgox7LShTB9i1vShaM2/file_1701890567783-407746_Screenshot_2023-11-20-10-53-55-423_com.facebook.katana.jpg', 'blockedProfiles': [

In [42]:
paragraph

["name: Arizza address: Panamá, , Panamá, Panama about_me:  age: 0 category: {'weight': 0, 'id': 'pf8IUMBw9rVBMGHoHdYw', 'text': 'user', 'slug': 'user', 'parentId': '', 'usedBy': 14, 'parentSlug': ''} tags: [{'weight': 0, 'id': 'pf8IUMBw9rVBMGHoHdYw', 'text': 'user', 'slug': 'user', 'parentId': '', 'usedBy': 14, 'parentSlug': ''}]",
 "name: Oriana Vega24422 address: PQ5V+3PR, PQ5V+3PR, Punto Fijo, Falcón, Venezuela about_me:  age: 27 category: {'weight': 0, 'id': 'pf8IUMBw9rVBMGHoHdYw', 'text': 'user', 'parentId': '', 'usedBy': 10, 'slug': 'user', 'parentSlug': ''} tags: [{'weight': 0, 'id': 'pf8IUMBw9rVBMGHoHdYw', 'text': 'user', 'parentId': '', 'usedBy': 10, 'slug': 'user', 'parentSlug': ''}, {'weight': 0, 'id': 'SCbBrQfQuPrGvu4H0o0d', 'text': 'musica', 'parentId': 'pf8IUMBw9rVBMGHoHdYw', 'usedBy': 0, 'slug': 'musica', 'parentSlug': 'user'}, {'weight': 0, 'id': 'UrUBAHe2qqD8nRI3GkUO', 'text': 'piano', 'parentId': 'pf8IUMBw9rVBMGHoHdYw', 'usedBy': 0, 'slug': 'piano', 'parentSlug': 'us

## Get data from Firebase

In [15]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
from google.cloud.firestore_v1.base_query import FieldFilter

from vertexai.generative_models import (
    FunctionDeclaration,
    GenerationConfig,
    GenerativeModel,
    Tool,
    Part,
    Content,
)

In [16]:
# Use a service account.
cred = credentials.Certificate("goblob-95e2a-6add9b68fd5d.json")
# cred = credentials.Certificate("aiuda-ffc77-b8546e446401.json")

app = firebase_admin.initialize_app(cred)

db = firestore.client()

ValueError: The default Firebase app already exists. This means you called initialize_app() more than once without providing an app name as the second argument. In most cases you only need to call initialize_app() once. But if you do want to initialize multiple apps, pass a second argument to initialize_app() to give each app a unique name.

In [17]:
profile = db.collection("profiles")
docs = profile.get()

# for doc in docs:
#    print(doc.to_dict())

In [18]:
doc.to_dict()

{'age': 32,
 'tags': [{'text': 'spa',
   'weight': 0,
   'id': 'jNDxsch7qIDIubx39R1K',
   'usedBy': 1,
   'parentSlug': '',
   'parentId': '',
   'slug': 'spa'},
  {'text': 'manos_y_pies',
   'weight': 0,
   'id': 'NjIAAjPlxDpQFsTTlHvp',
   'usedBy': 0,
   'parentSlug': 'spa',
   'parentId': 'jNDxsch7qIDIubx39R1K',
   'slug': 'manos_y_pies'}],
 'displayName': 'Emulator 66980917',
 'commentsCount': 0,
 'realTimeLocEnabled': False,
 'network': [],
 'rating': 0,
 'avgRating': 4.5,
 'about': '',
 'socialNetworks': [],
 'userId': 'ogey2VejTNQ4q3HnIeQ5p5veipo1',
 'totalRatingVotes': 0,
 'blockedProfiles': [],
 'service': {'text': 'spa',
  'weight': 0,
  'id': 'jNDxsch7qIDIubx39R1K',
  'usedBy': 1,
  'parentSlug': '',
  'parentId': '',
  'slug': 'spa'},
 'address': '132, Calle C, San Miguelito, Provincia de Panamá, Panama',
 'gender': 'female',
 'photo': '',
 'location': <google.cloud.firestore_v1._helpers.GeoPoint at 0x16d2f2bc0>,
 'totalReviews': 2}

In [19]:
profiles = []
for doc in docs:
    
    user_profile = doc.to_dict()

    name = user_profile["displayName"]
    address = user_profile["address"] if "address" in user_profile.keys() is not None else ""
    gender = user_profile["gender"]
    age = user_profile["age"]
    # rating = user_profile["avgRating"]
    tags = user_profile["tags"]
    service = user_profile["service"]
    about = user_profile["about"]

    if user_profile["location"] is not None:
        location = {
            "lat": docs[0].to_dict()["location"].latitude,
            "long": docs[0].to_dict()["location"].longitude,
        }

    print(name, location)

    try:
        profiles.append("name: " + name + " address: " + address + " about_me: " + about + " age: " + str(age) + " category: " + str(service) + " tags: " + str(tags) + \
                    " gender: " + gender + " location: " + str(location))
    except:
        print("error")

Violeta IOS {'lat': 8.97548104431072, 'long': -79.50703719035977}
violeta ios 0.7.17 {'lat': 8.97548104431072, 'long': -79.50703719035977}
Arizza {'lat': 8.97548104431072, 'long': -79.50703719035977}
Oriana Vega24422 {'lat': 8.97548104431072, 'long': -79.50703719035977}
Alejandro  {'lat': 8.97548104431072, 'long': -79.50703719035977}
Jules D. {'lat': 8.97548104431072, 'long': -79.50703719035977}
Shiro {'lat': 8.97548104431072, 'long': -79.50703719035977}
Daniela Febles {'lat': 8.97548104431072, 'long': -79.50703719035977}
Akshdeep {'lat': 8.97548104431072, 'long': -79.50703719035977}
Harry {'lat': 8.97548104431072, 'long': -79.50703719035977}
Manjeet kaur {'lat': 8.97548104431072, 'long': -79.50703719035977}
YoMap Assistant {'lat': 8.97548104431072, 'long': -79.50703719035977}
Jino  {'lat': 8.97548104431072, 'long': -79.50703719035977}
YoMap Assistant {'lat': 8.97548104431072, 'long': -79.50703719035977}
Ricardo De Leon {'lat': 8.97548104431072, 'long': -79.50703719035977}
shankar {'la

In [20]:
profiles

["name: Violeta IOS address: Calle Punta Colón, Calle Punta Colón, Panama City, Provincia de Panamá, Panamá about_me: Guardo y paseo de mascotas. Soy estudiante 3er ano escuela veterinaria de Panama age: 29 category: {'text': 'pets', 'weight': 0, 'id': 'qDwGRcWqjkBpjX4rsDm0', 'usedBy': 0, 'parentSlug': '', 'parentId': '', 'slug': 'pets'} tags: [{'text': 'pets', 'weight': 0, 'id': 'qDwGRcWqjkBpjX4rsDm0', 'usedBy': 0, 'parentSlug': '', 'parentId': '', 'slug': 'pets'}, {'text': 'paseo_de_perros', 'weight': 0, 'id': 'x6gzflGmUYKkYklPtJcp', 'usedBy': 0, 'parentSlug': 'pets', 'parentId': 'qDwGRcWqjkBpjX4rsDm0', 'slug': 'paseo_de_perros'}, {'text': 'dog_sitting', 'weight': 0, 'id': 'YQlpTiiGugujzP1ZmyOT', 'usedBy': 0, 'parentSlug': 'pets', 'parentId': 'qDwGRcWqjkBpjX4rsDm0', 'slug': 'dog_sitting'}] gender: female location: {'lat': 8.97548104431072, 'long': -79.50703719035977}",
 "name: violeta ios 0.7.17 address:  about_me: ¡Hola! Soy Violeta, una apasionada por los animales.  con  años de ex

## Send data to Neo4j

In [21]:
def set_nodes_to_graph(text):

    # Connect to the Neo4j database
    driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

    # List the entities and relations the LLM should look for in the text
    entities = ["Person", "Category", "City", "Tags"]
    relations = ["CATEGORY_PROVIDER", "FROM_CITY", "TAGS"]
    potential_schema = [
        ("Person", "CATEGORY_PROVIDER", "Category"),
        ("Person", "FROM_CITY", "City"),
        ("Person", "TAGS", "Tags"),
    ]

    # Create an Embedder object
    embedder = OpenAIEmbeddings(model="text-embedding-3-large")

    # Instantiate the LLM
    llm = OpenAILLM(
        model_name="gpt-4o",
        model_params={
            "max_tokens": 2000,
            "response_format": {"type": "json_object"},
            "temperature": 0,
        },
    )

    # Instantiate the SimpleKGPipeline
    kg_builder = SimpleKGPipeline(
        llm=llm,
        driver=driver,
        embedder=embedder,
        entities=entities,
        relations=relations,
        potential_schema=potential_schema,
        on_error="IGNORE",
        from_pdf=False,
    )

    # Run the pipeline on a piece of text
    asyncio.run(kg_builder.run_async(text=text))
    driver.close()

In [22]:
for item in profiles:
    set_nodes_to_graph(item)

In [65]:
from neo4j.spatial import CartesianPoint

point = CartesianPoint((1.23, 4.56))
print(point.x, point.y, point.srid)
# 1.23 4.56 7203
from neo4j.spatial import CartesianPoint

point = CartesianPoint((1.23, 4.56, 7.89))
print(point.x, point.y, point.z, point.srid)

1.23 4.56 7203
1.23 4.56 7.89 9157


In [27]:
from neo4j import GraphDatabase

# Neo4j connection credentials
NEO4J_URI = "bolt://localhost:7687"  # Update with your Neo4j URI
USERNAME = "neo4j"
PASSWORD = "cassandra"

# Sample location data (latitude, longitude)
DEFAULT_LATITUDE = 40.7128   # Example: New York City
DEFAULT_LONGITUDE = -74.0060

# Neo4j update query
UPDATE_QUERY = """
MATCH (p:Person)
SET p.location = point({latitude: $latitude, longitude: $longitude})
RETURN COUNT(p) AS updated_count;
"""

# Function to update a person's location
def update_person_location_by_name(person_name, latitude, longitude):
    UPDATE_QUERY = """
    MATCH (p:Person {name: $name})
    SET p.location = point({latitude: $latitude, longitude: $longitude})
    RETURN p.name AS updated_name, p.location AS new_location;
    """

    # Connect to Neo4j
    driver = GraphDatabase.driver(NEO4J_URI, auth=(USERNAME, PASSWORD))

    with driver.session() as session:
        result = session.run(UPDATE_QUERY, name=person_name, latitude=latitude, longitude=longitude)
        record = result.single()
        if record:
            print(f"✅ Updated location for {record['updated_name']} to {record['new_location']}")
        else:
            print(f"❌ No person found with name '{person_name}'")

    driver.close()

In [28]:
for doc in docs:
    
    user_profile = doc.to_dict()

    try:
        if "location" in user_profile.keys():
            location = {
                "lat": doc.to_dict()["location"].latitude,
                "long": doc.to_dict()["location"].longitude,
            }

            update_person_location_by_name(user_profile["displayName"], location['lat'], location['long'])
    except:
        print(doc)

✅ Updated location for Violeta IOS to POINT(-79.50703719035977 8.97548104431072)
❌ No person found with name 'violeta ios 0.7.17'
✅ Updated location for Arizza to POINT(-79.55968186016943 8.994300842285156)
✅ Updated location for Oriana Vega24422 to POINT(-70.2067349 11.7080935)
❌ No person found with name 'Alejandro '
✅ Updated location for Jules D. to POINT(2.3113472 48.8598247)
✅ Updated location for Shiro to POINT(-79.5110741 8.9853828)
✅ Updated location for Daniela Febles to POINT(-79.51779272414699 8.985404968261719)
✅ Updated location for Akshdeep to POINT(75.0192061 31.0978)
✅ Updated location for Harry to POINT(135.9516219 33.6266539)
✅ Updated location for Manjeet kaur to POINT(75.2686158 29.5766614)
❌ No person found with name 'Jino '
✅ Updated location for YoMap Assistant to POINT(-79.5046593 8.9898528)
✅ Updated location for Ricardo De Leon to POINT(-79.5271224 8.9791855)
✅ Updated location for shankar to POINT(72.862981 19.1749283)
❌ No person found with name 'joginder s

In [79]:
location['lat']

9.03795683773441

In [30]:
INDEX_NAME

'yomap-index'

In [49]:
from neo4j import GraphDatabase
from neo4j_graphrag.embeddings import OpenAIEmbeddings
from neo4j_graphrag.generation import GraphRAG
from neo4j_graphrag.llm import OpenAILLM
from neo4j_graphrag.retrievers import VectorRetriever

# Connect to the Neo4j database
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

# Create an Embedder object
embedder = OpenAIEmbeddings(model="text-embedding-3-large")

# Initialize the retriever
retriever = VectorRetriever(driver, "yomap_index", embedder)

# Instantiate the LLM
llm = OpenAILLM(model_name="gpt-4o", model_params={"temperature": 0})

# Instantiate the RAG pipeline
rag = GraphRAG(retriever=retriever, llm=llm)

In [50]:
# Query the graph
query_text = "Algun paseador de perros en la base de datos?"
response = rag.search(query_text=query_text, retriever_config={"top_k": 5})
print(response.answer)
driver.close()

Sí, en la base de datos hay varias personas que ofrecen servicios de paseo de perros:

1. Violeta IOS (35 años) - Apasionada por los animales, ofrece cuidado y paseos de perros.
2. Andres Laguna (25 años) - Ofrece adiestramiento canino y paseos caninos.
3. Violeta IOS (29 años) - Estudiante de veterinaria, guarda y pasea mascotas.
4. Readet Ferche (36 años) - Ofrece servicios de paseo de perros y cuidado experto.


In [51]:
# Query the graph
query_text = "alguna provider cerca de latitude: 8.96009198, longitude:  -79.538242 y que sea paseador de perros y me dices a que distancia esta de esta location"
response = rag.search(query_text=query_text, retriever_config={"top_k": 5})
print(response.answer)
driver.close()

Para encontrar un proveedor cerca de la latitud 8.96009198 y longitud -79.538242 que ofrezca servicios de paseo de perros, podemos revisar los datos proporcionados:

1. **Andres Laguna**: Ofrece servicios de adiestramiento canino y paseos caninos. Su ubicación es en la latitud 8.97548104431072 y longitud -79.50703719035977.

2. **Violeta IOS**: Ofrece servicios de paseo de perros y cuidado de mascotas. Su ubicación es en la latitud 8.97548104431072 y longitud -79.50703719035977.

3. **Readet Ferche**: Ofrece servicios de paseo de perros. Su ubicación es en la latitud 8.97548104431072 y longitud -79.50703719035977.

4. **Higal Ramal**: Ofrece servicios relacionados con mascotas, incluyendo cuidado de perros. Su ubicación es en la latitud 8.97548104431072 y longitud -79.50703719035977.

Para calcular la distancia entre la ubicación proporcionada y la de los proveedores, podemos usar la fórmula de la distancia euclidiana en un plano cartesiano. Sin embargo, dado que todas las ubicaciones 

In [102]:
from langchain_neo4j import Neo4jVector

In [106]:
store = Neo4jVector.from_existing_index(
    embedder,
    url=NEO4J_URI, 
    username=NEO4J_USERNAME, 
    password=NEO4J_PASSWORD,
    index_name=INDEX_NAME,
)

In [110]:
query = "Algun paseador de perros en la base de datos?"

docs_with_score = store.similarity_search_with_score(query, k=5)

In [111]:
for doc, score in docs_with_score:
    print("-" * 80)
    print("Score: ", score)
    print(doc.page_content)
    print("-" * 80)

--------------------------------------------------------------------------------
Score:  0.5258879065513611
name: violeta ios 0.7.17 address:  about_me: ¡Hola! Soy Violeta, una apasionada por los animales.  con  años de experiencia. Amo cuidar de perros y pasearlos. Si buscas una persona confiable y cariñosa para tu peludo amigo, ¡contacta conmigo! age: 35 category: {'text': 'pets', 'weight': 0, 'id': 'qDwGRcWqjkBpjX4rsDm0', 'parentSlug': '', 'usedBy': 5, 'parentId': '', 'slug': 'pets'} tags: [{'text': 'pets', 'weight': 0, 'id': 'qDwGRcWqjkBpjX4rsDm0', 'parentSlug': '', 'usedBy': 5, 'parentId': '', 'slug': 'pets'}, {'text': 'dog_sitting', 'weight': 0, 'id': 'YQlpTiiGugujzP1ZmyOT', 'parentSlug': 'pets', 'usedBy': 5, 'parentId': 'qDwGRcWqjkBpjX4rsDm0', 'slug': 'dog_sitting'}, {'text': 'paseo_de_perros', 'weight': 0, 'id': 'x6gzflGmUYKkYklPtJcp', 'parentSlug': 'pets', 'usedBy': 2, 'parentId': 'qDwGRcWqjkBpjX4rsDm0', 'slug': 'paseo_de_perros'}] gender: female location: {'lat': 8.9754810443

In [115]:
query = """
MATCH (c:Person) 

WHERE point.distance(c.location, point({latitude: 8.96009198, longitude:  -79.538242})) / 1000 < 2

RETURN c.name, point.distance(c.location, point({latitude:8.96009198, longitude:  -79.538242})) / 1000
"""

store.query(query)

[{'c.name': 'Alex',
  'point.distance(c.location, point({latitude:8.96009198, longitude:  -79.538242})) / 1000': 8.841852492993769e-05},
 {'c.name': 'Laissa',
  'point.distance(c.location, point({latitude:8.96009198, longitude:  -79.538242})) / 1000': 1.9547561527421844}]